In [ ]:
%%time
%run -i 'nlp_implementation.py' #imports packages and inits alltext
pod_name = "Lex_Fridman_Podcast1"

onlyfiles = [(text_fix(open(pod_name+"/"+f).read()), f) for f in listdir(pod_name) if isfile(join(pod_name, f))]#list of all podcast files

doc_bin = DocBin(store_user_data=True) #docbin container for serialization

docs = [] #list of docs 
print("start")
for doc, name in nlp.pipe(onlyfiles, as_tuples=True): #piping all collection of docs
    
    name = re.split("[.\|]",name)[:-1]
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = "Lex Fridman"
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["topic"]= str(name[1]).replace("_"," ")
    doc.user_data["entis"] = [(ent.text, ent.label_) for ent in doc.ents]
    docs.append(doc)
    doc_bin.add(doc) #add doc to list and bin

print(docs[0].user_data["host"])

In [ ]:
with open("cucumber/first_doc.bin", "wb") as f: #write bytestream to disk
    f.write(doc_bin.to_bytes())
print("done")

In [ ]:
%%time ##loading in bytestreams again
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("en_core_web_lg")
with open("cucumber/all_doc.bin", "rb") as w: #loading in
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))